In [4]:
import numpy as np
import pandas as pd
import os
import math
from time import time
#定义几个函数
# def find_key(keys,obj):  二分查找user id为obj的用户是否在训练集用户列表keys中出现
# def softmax(vec): 9类别的时长转softmax概率，时长太大，指数溢出暂时不用
# def cnt2pro(vec): 9类别的时长转概率，分别除以9类别时长总和
# def close_txt(id_cnt_files): 一次性关闭多个txt文件，id_cnt_files为文件列表
# def getfiles(mode): 返回文件名列表，mode："train", "val", "test"

def find_key(keys,obj):
    len_keys=len(keys)
    l=0
    r=len_keys-1
    while True:
        if r-l<3:
            break
        mid=(l+r)//2
        if keys[mid]==obj:
            return True
        if keys[mid]<obj:
            l=mid+1
        else:
            r=mid+1
    for k in range(l,r+1):
        if keys[k]==obj:
            return True
    return False    
def softmax(vec):
    return_ans=np.zeros([vec.shape[0]])
    for k in range(vec.shape[0]):
        SUM=0.0
        for n in range(vec.shape[0]):
            SUM+=math.exp(vec[n]-vec[k])
        return_ans=1.0/SUM
    return return_ans
def cnt2pro(vec):
    return_ans=np.zeros([vec.shape[0]])
    SUM=0.0
    for n in range(vec.shape[0]):
        SUM+=vec[n]
    for k in range(vec.shape[0]):
        return_ans[k]=vec[k]/SUM
    return return_ans
def close_txt(id_cnt_files):
    for file in id_cnt_files:
        file.close()
def getfiles(mode):
    #嵌套函数
    def get_name_from_txt(file_name):
        name_id=[]
        f=open(file_name)
        lines=f.readlines()
        for line in lines:
            s=line.split(".jpg")[0]
            s=s.split("/")[-1]
            name_id.append(s)
        f.close()
        return name_id
    ##
    return_ans=[]
    if mode=="test":
        for i in range(2):
            path="/root/userfolder/luotao/final/test_visit_"+str(i)+"/"+str(i)+"/"
            files=os.listdir(path)
            for file in files:
                return_ans.append(path+file)
        return_ans.sort()
        return return_ans
    elif mode=="train":
        file_name="/root/userfolder/luotao/final_code/data_txt/train.txt"
    elif mode=="val":
        file_name="/root/userfolder/luotao/final_code/data_txt/val.txt"
    name_id=get_name_from_txt(file_name)
    name2file={}
    for i in range(10):
        path="/root/userfolder/luotao/final/train_visit_"+str(i)+"/"+str(i)+"/"
        files=os.listdir(path)
        for file in files:
            s=file.split(".txt")[0]
            name2file[s]=path+file
    return_ans=[]
    for name in name_id:
        return_ans.append(name2file[name])
    return return_ans

In [31]:
# 对训练集进行处理的代码
# 遍历训练集的原始visit txt文件，分9个类别记录每个用户的总时长
# 输出9个txt，每个文件对应一个类别，每个文件中两列记录用户id和时长
# 分两步进行：第一步：遍历txt，第二步：对于每个类别，依据时长对用户排序，删除时长低的80% 或 90% 的用户，再合并同一类别中的相同用户，时长相加

###################################################第一步
id_cnt_files=[]
for i in range(1,10):
    id_cnt_file=open("id_cnt/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
files=getfiles("train")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    label=int(file.split(".txt")[0].split("/")[-1].split("_")[1])
    table = pd.read_table(file, header=None)
    tv=table.values
    user_num=tv.shape[0]
    temp=""
    for i in range(user_num):
        temp+=str(tv[i,0])+","+str(tv[i,1].count("&")) +"\n"
    id_cnt_files[label-1].write(temp)
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1
close_txt(id_cnt_files)

350000
0.0000% 0.004881143569946289
0.2857% 54.52988576889038
0.5714% 67.98733687400818
0.8571% 55.748059034347534
1.1429% 56.88954448699951
1.4286% 51.62448287010193
1.7143% 54.97791314125061
2.0000% 53.61399984359741
2.2857% 54.15880799293518
2.5714% 52.98566508293152
2.8571% 53.19744658470154
3.1429% 56.51144051551819
3.4286% 58.554917097091675
3.7143% 68.3220887184143
4.0000% 70.19179630279541
4.2857% 64.68568825721741
4.5714% 70.81199312210083
4.8571% 69.83177995681763
5.1429% 70.26169633865356
5.4286% 68.64315748214722
5.7143% 68.92847895622253
6.0000% 67.32691049575806
6.2857% 69.83403635025024
6.5714% 66.91099977493286
6.8571% 65.37427425384521
7.1429% 65.46290469169617
7.4286% 65.7268636226654
7.7143% 59.82199835777283
8.0000% 66.35263729095459
8.2857% 62.591543674468994
8.5714% 67.45076608657837
8.8571% 68.56491446495056
9.1429% 70.49326229095459
9.4286% 78.3373076915741
9.7143% 67.16533088684082
10.0000% 69.6611680984497
10.2857% 69.37723708152771
10.5714% 68.82330465316772


KeyboardInterrupt: 

In [10]:
import numpy as np
import pandas as pd
import os
import math
from time import time
###################################################第二步
id_cnt_files=[]
for i in range(1,10):
    id_cnt_file=open("id_cnt_del_same_0.9_0.9/id_cnt_"+str(i)+".txt","a+")
    id_cnt_files.append(id_cnt_file)
    
last_time=time()
for i in range(1,10):
    table = pd.read_table("id_cnt/id_cnt_new_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0.9
#     if i in [1,2,7,9]:
#         rate=0.95
#     else:
#         rate=0.85
    first_user=int(user_num*rate)
    user2cnt={}
    last_user=""
    for j in range(first_user,user_num):
        if (j-first_user)%10000==0:
            now_time=time()
            print(i,(j-first_user)/(user_num-first_user),now_time-last_time)
            last_time=time()
        now_user=sorted_data[j,0]
        if last_user==now_user:
            user2cnt[now_user]+=sorted_data[j,1]
        else:
            user2cnt[now_user]=sorted_data[j,1]
            last_user=now_user
    users=user2cnt.keys()
    for user in users:
        id_cnt_files[i-1].write(str(user)+","+str(user2cnt[user])+"\n")
close_txt(id_cnt_files)

1 0.0 1069.4107112884521
1 0.0009409618869974405 0.015664339065551758
1 0.001881923773994881 0.021217823028564453
1 0.0028228856609923214 0.03675651550292969
1 0.003763847547989762 0.02393198013305664
1 0.004704809434987202 0.04139399528503418
1 0.005645771321984643 0.03150749206542969
1 0.006586733208982083 0.018470287322998047
1 0.007527695095979524 0.014098644256591797
1 0.008468656982976965 0.014215946197509766
1 0.009409618869974405 0.016259431838989258
1 0.010350580756971846 0.013042688369750977
1 0.011291542643969286 0.011992216110229492
1 0.012232504530966727 0.012816667556762695
1 0.013173466417964167 0.02041459083557129
1 0.014114428304961608 0.02843642234802246
1 0.015055390191959048 0.032466888427734375
1 0.01599635207895649 0.045015811920166016
1 0.01693731396595393 0.059359073638916016
1 0.01787827585295137 0.04082632064819336
1 0.01881923773994881 0.012554407119750977
1 0.01976019962694625 0.012205839157104492
1 0.020701161513943692 0.012217283248901367
1 0.0216421234009

1 0.181605644190506 0.07166481018066406
1 0.18254660607750345 0.023987293243408203
1 0.1834875679645009 0.013498544692993164
1 0.18442852985149832 0.012739896774291992
1 0.1853694917384958 0.01647782325744629
1 0.18631045362549323 0.01287221908569336
1 0.18725141551249067 0.012922525405883789
1 0.1881923773994881 0.014164209365844727
1 0.18913333928648554 0.013075113296508789
1 0.19007430117348298 0.012966632843017578
1 0.19101526306048042 0.01282644271850586
1 0.19195622494747785 0.012911319732666016
1 0.1928971868344753 0.01348567008972168
1 0.19383814872147273 0.013222694396972656
1 0.19477911060847017 0.012668371200561523
1 0.19572007249546763 0.01287531852722168
1 0.19666103438246507 0.012479305267333984
1 0.1976019962694625 0.012714147567749023
1 0.19854295815645995 0.012308597564697266
1 0.19948392004345739 0.012715816497802734
1 0.20042488193045482 0.012666463851928711
1 0.20136584381745226 0.013956069946289062
1 0.2023068057044497 0.018823862075805664
1 0.20324776759144714 0.0

1 0.37450283102498133 0.06074166297912598
1 0.37544379291197877 0.03481101989746094
1 0.3763847547989762 0.01927018165588379
1 0.37732571668597364 0.017812728881835938
1 0.3782666785729711 0.013465404510498047
1 0.3792076404599685 0.01956033706665039
1 0.38014860234696596 0.01634955406188965
1 0.3810895642339634 0.019776582717895508
1 0.38203052612096083 0.02915477752685547
1 0.38297148800795827 0.013856887817382812
1 0.3839124498949557 0.013465642929077148
1 0.38485341178195315 0.013725042343139648
1 0.3857943736689506 0.028562545776367188
1 0.386735335555948 0.0938878059387207
1 0.38767629744294546 0.020081758499145508
1 0.3886172593299429 0.02238774299621582
1 0.38955822121694034 0.018152713775634766
1 0.39049918310393783 0.020430803298950195
1 0.39144014499093527 0.019725799560546875
1 0.3923811068779327 0.01383829116821289
1 0.39332206876493014 0.013739585876464844
1 0.3942630306519276 0.012957096099853516
1 0.395203992538925 0.013409614562988281
1 0.39614495442592246 0.0215406417

1 0.5645771321984643 0.044095516204833984
1 0.5655180940854617 0.01423192024230957
1 0.5664590559724592 0.013595104217529297
1 0.5674000178594566 0.013608932495117188
1 0.568340979746454 0.012767314910888672
1 0.5692819416334515 0.011892080307006836
1 0.5702229035204489 0.011790275573730469
1 0.5711638654074463 0.011750459671020508
1 0.5721048272944438 0.011600971221923828
1 0.5730457891814412 0.012229442596435547
1 0.5739867510684387 0.013280868530273438
1 0.5749277129554361 0.028249263763427734
1 0.5758686748424335 0.04498577117919922
1 0.576809636729431 0.04009294509887695
1 0.5777505986164284 0.029188156127929688
1 0.5786915605034259 0.02098250389099121
1 0.5796325223904233 0.021879196166992188
1 0.5805734842774208 0.016358375549316406
1 0.5815144461644183 0.014488935470581055
1 0.5824554080514157 0.01452016830444336
1 0.5833963699384132 0.013338327407836914
1 0.5843373318254106 0.013367414474487305
1 0.585278293712408 0.013527154922485352
1 0.5862192555994055 0.013800621032714844


1 0.7565333571459422 0.013258695602416992
1 0.7574743190329396 0.012968063354492188
1 0.758415280919937 0.013420820236206055
1 0.7593562428069345 0.01381373405456543
1 0.7602972046939319 0.022321701049804688
1 0.7612381665809294 0.023656845092773438
1 0.7621791284679268 0.02323317527770996
1 0.7631200903549242 0.022641420364379883
1 0.7640610522419217 0.019426345825195312
1 0.7650020141289191 0.019890308380126953
1 0.7659429760159165 0.019815444946289062
1 0.766883937902914 0.01917123794555664
1 0.7678248997899114 0.016526460647583008
1 0.7687658616769089 0.013654232025146484
1 0.7697068235639063 0.013097047805786133
1 0.7706477854509037 0.013103485107421875
1 0.7715887473379012 0.012706279754638672
1 0.7725297092248986 0.012414693832397461
1 0.773470671111896 0.01245427131652832
1 0.7744116329988935 0.013047456741333008
1 0.7753525948858909 0.013727664947509766
1 0.7762935567728884 0.025364160537719727
1 0.7772345186598858 0.03515744209289551
1 0.7781754805468832 0.07404899597167969
1

...

7 0.4003036404572316 0.039353370666503906
7 0.4021483576943617 0.03869795799255371
7 0.40399307493149184 0.015529155731201172
7 0.4058377921686219 0.01795196533203125
7 0.407682509405752 0.01717829704284668
7 0.4095272266428821 0.023561954498291016
7 0.4113719438800122 0.022960186004638672
7 0.41321666111714234 0.02160811424255371
7 0.4150613783542724 0.021284103393554688
7 0.4169060955914025 0.02010512351989746
7 0.4187508128285326 0.017656326293945312
7 0.4205955300656627 0.02065873146057129
7 0.42244024730279284 0.021151304244995117
7 0.4242849645399229 0.020043134689331055
7 0.426129681777053 0.020342111587524414
7 0.4279743990141831 0.022853612899780273
7 0.4298191162513132 0.024356842041015625
7 0.4316638334884433 0.02870011329650879
7 0.4335085507255734 0.045142412185668945
7 0.4353532679627035 0.020108461380004883
7 0.4371979851998336 0.014815330505371094
7 0.4390427024369637 0.012798786163330078
7 0.4408874196740938 0.012459516525268555
7 0.44273213691122393 0.0127375125885009

7 0.7637129361718612 0.03609061241149902
7 0.7655576534089913 0.029438018798828125
7 0.7674023706461215 0.025163888931274414
7 0.7692470878832516 0.061676025390625
7 0.7710918051203817 0.018597841262817383
7 0.7729365223575118 0.014167070388793945
7 0.7747812395946418 0.014417648315429688
7 0.7766259568317719 0.013139963150024414
7 0.778470674068902 0.013139009475708008
7 0.7803153913060321 0.013497114181518555
7 0.7821601085431622 0.016182422637939453
7 0.7840048257802923 0.01515650749206543
7 0.7858495430174224 0.015910625457763672
7 0.7876942602545526 0.0160067081451416
7 0.7895389774916827 0.023747682571411133
7 0.7913836947288128 0.025439739227294922
7 0.7932284119659428 0.02952265739440918
7 0.7950731292030729 0.05522489547729492
7 0.796917846440203 0.04854631423950195
7 0.7987625636773331 0.02366042137145996
7 0.8006072809144632 0.013483762741088867
7 0.8024519981515933 0.012876033782958984
7 0.8042967153887234 0.012830257415771484
7 0.8061414326258536 0.01274728775024414
7 0.80

8 0.6513151980273406 0.012517690658569336
8 0.6604886515206834 0.012084484100341797
8 0.6696621050140262 0.011809825897216797
8 0.678835558507369 0.011530160903930664
8 0.6880090120007119 0.011555671691894531
8 0.6971824654940547 0.012880802154541016
8 0.7063559189873975 0.01202845573425293
8 0.7155293724807403 0.011803865432739258
8 0.7247028259740832 0.011580228805541992
8 0.733876279467426 0.011904716491699219
8 0.7430497329607688 0.01224517822265625
8 0.7522231864541117 0.01751089096069336
8 0.7613966399474544 0.02109980583190918
8 0.7705700934407973 0.04385876655578613
8 0.77974354693414 0.04633212089538574
8 0.7889170004274829 0.02507185935974121
8 0.7980904539208258 0.07573080062866211
8 0.8072639074141685 0.012356281280517578
8 0.8164373609075114 0.011897087097167969
8 0.8256108144008543 0.011795759201049805
8 0.834784267894197 0.011836767196655273
8 0.8439577213875399 0.011738777160644531
8 0.8531311748808827 0.011854887008666992
8 0.8623046283742255 0.011887788772583008
8 0.8

9 0.3445365486695922 0.012880325317382812
9 0.34665026982707436 0.01628899574279785
9 0.3487639909845565 0.012384176254272461
9 0.35087771214203867 0.013139963150024414
9 0.3529914332995208 0.012336015701293945
9 0.355105154457003 0.016872167587280273
9 0.3572188756144852 0.019264936447143555
9 0.3593325967719673 0.012176275253295898
9 0.3614463179294495 0.01636981964111328
9 0.36356003908693163 0.012872695922851562
9 0.3656737602444138 0.012017250061035156
9 0.367787481401896 0.011871337890625
9 0.36990120255937814 0.01197052001953125
9 0.3720149237168603 0.012017011642456055
9 0.37412864487434244 0.012119293212890625
9 0.3762423660318246 0.013021230697631836
9 0.37835608718930674 0.01368260383605957
9 0.38046980834678895 0.0152740478515625
9 0.3825835295042711 0.016116619110107422
9 0.38469725066175325 0.01672506332397461
9 0.3868109718192354 0.014493227005004883
9 0.38892469297671756 0.012379884719848633
9 0.3910384141341997 0.012277841567993164
9 0.3931521352916819 0.01294398307800

9 0.7736219436384708 0.013485431671142578
9 0.775735664795953 0.012807607650756836
9 0.7778493859534351 0.012836456298828125
9 0.7799631071109173 0.013727426528930664
9 0.7820768282683994 0.013458251953125
9 0.7841905494258816 0.012514829635620117
9 0.7863042705833638 0.014566421508789062
9 0.7884179917408459 0.013397693634033203
9 0.7905317128983281 0.01336050033569336
9 0.7926454340558102 0.015102148056030273
9 0.7947591552132924 0.013353824615478516
9 0.7968728763707746 0.01327204704284668
9 0.7989865975282567 0.012879610061645508
9 0.8011003186857389 0.012569427490234375
9 0.803214039843221 0.013009786605834961
9 0.8053277610007032 0.01243138313293457
9 0.8074414821581853 0.012321949005126953
9 0.8095552033156675 0.012838363647460938
9 0.8116689244731498 0.013560771942138672
9 0.8137826456306319 0.012386798858642578
9 0.8158963667881141 0.013429880142211914
9 0.8180100879455962 0.013916015625
9 0.8201238091030784 0.013283729553222656
9 0.8222375302605606 0.012481689453125
9 0.82435

In [5]:
# 测试代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_table("id_cnt_del_same/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        now_user=sorted_data[j,0]
        user2hours[i-1][now_user]=int(sorted_data[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

id2label=np.zeros([100000])
id2pro=np.zeros([100000,9])
id2num=np.zeros([100000,9])
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("test")
all_cnt=len(files)
print(all_cnt)
cnt=0
last_time=time()
for file in files:
    reg_id=int(file.split(".txt")[0].split("/")[-1])
    table = pd.read_table(file, header=None)
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        now_user_hours=table2cnt[i,1].count("&")
        table2cnt[i,1]=now_user_hours
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0.5   #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user):
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label
    id2num[reg_id]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[reg_id]=pre_label
    id2pro[reg_id]=pro[:]
    if cnt%1000==0:
        now_time=time()
        print("%.4f%%"%(cnt/all_cnt*100),now_time-last_time)
        last_time=time()
    cnt+=1

np.save("id2num_test.npy",id2num)
np.save("id2label_test.npy",id2label)
np.save("id2pro_test.npy",id2pro)

1 0.0
1 0.005442423890150987
1 0.010884847780301974
1 0.01632727167045296
1 0.021769695560603947
1 0.027212119450754933
1 0.03265454334090592
1 0.03809696723105691
1 0.043539391121207895
1 0.048981815011358884
1 0.05442423890150987
1 0.059866662791660856
1 0.06530908668181185
1 0.07075151057196283
1 0.07619393446211382
1 0.0816363583522648
1 0.08707878224241579
1 0.09252120613256679
1 0.09796363002271777
1 0.10340605391286875
1 0.10884847780301973
1 0.11429090169317073
1 0.11973332558332171
1 0.1251757494734727
1 0.1306181733636237
1 0.13606059725377467
1 0.14150302114392566
1 0.14694544503407664
1 0.15238786892422765
1 0.15783029281437863
1 0.1632727167045296
1 0.1687151405946806
1 0.17415756448483158
1 0.17959998837498256
1 0.18504241226513357
1 0.19048483615528455
1 0.19592726004543554
1 0.20136968393558652
1 0.2068121078257375
1 0.21225453171588848
1 0.21769695560603947
1 0.22313937949619048
1 0.22858180338634146
1 0.23402422727649244
1 0.23946665116664342
1 0.2449090750567944
1 0.

5 0.8124235814068739
6 0.0
6 0.014578045754654406
6 0.029156091509308812
6 0.04373413726396322
6 0.058312183018617625
6 0.07289022877327203
6 0.08746827452792644
6 0.10204632028258084
6 0.11662436603723525
6 0.13120241179188966
6 0.14578045754654406
6 0.16035850330119847
6 0.17493654905585287
6 0.18951459481050728
6 0.2040926405651617
6 0.2186706863198161
6 0.2332487320744705
6 0.2478267778291249
6 0.2624048235837793
6 0.2769828693384337
6 0.2915609150930881
6 0.3061389608477425
6 0.32071700660239694
6 0.3352950523570513
6 0.34987309811170575
6 0.3644511438663601
6 0.37902918962101456
6 0.39360723537566894
6 0.4081852811303234
6 0.42276332688497775
6 0.4373413726396322
6 0.45191941839428657
6 0.466497464148941
6 0.4810755099035954
6 0.4956535556582498
6 0.5102316014129042
6 0.5248096471675586
6 0.539387692922213
6 0.5539657386768674
6 0.5685437844315219
6 0.5831218301861762
6 0.5976998759408306
6 0.612277921695485
6 0.6268559674501394
6 0.6414340132047939
6 0.6560120589594483
6 0.67059

/root/userfolder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


1.0000% 58.33874249458313
2.0000% 68.29348754882812
3.0000% 59.736905097961426
4.0000% 68.2861065864563
5.0000% 68.64053511619568
6.0000% 66.26126027107239
7.0000% 66.01011419296265
8.0000% 60.4229941368103
9.0000% 69.19461297988892
10.0000% 56.27913546562195
11.0000% 74.5817883014679
12.0000% 100.00872683525085
13.0000% 81.46762156486511
14.0000% 88.88757920265198
15.0000% 88.55194926261902
16.0000% 54.971025466918945
17.0000% 60.24658179283142
18.0000% 67.28697109222412
19.0000% 82.02545499801636
20.0000% 84.38119864463806
21.0000% 81.23060941696167
22.0000% 82.29792976379395
23.0000% 83.7995171546936
24.0000% 86.35583162307739
25.0000% 86.3526840209961
26.0000% 75.70232915878296
27.0000% 55.94208645820618
28.0000% 62.01705980300903
29.0000% 62.695639848709106
30.0000% 132.43969225883484
31.0000% 136.69162368774414
32.0000% 127.06249022483826
33.0000% 127.35391068458557
34.0000% 88.90705013275146
35.0000% 76.59721970558167
36.0000% 74.59156513214111
37.0000% 154.62042808532715
38.000

In [12]:
# 验证集代码
##################################################从id_cnt_del_same文件夹读取训练集的用户时长，存入9个字典中 user2hours
user2hours=[{} for i in range(9)]
for i in range(1,10):
    label=i
    table = pd.read_table("result_del[if 1.2.7.9 then 0.9 else 0.8]_test[0.5]/id_cnt_del_same_0.9_0.8/id_cnt_"+str(i)+".txt",sep=",", header=None)    
    temp=table.values[:,:]
    user_num=temp.shape[0]
    sorted_data=temp[np.lexsort(temp.T)]
    for j in range(user_num):
        if (j)%50000==0:
            print(i,(j)/(user_num))
        now_user=sorted_data[j,0]
        user2hours[i-1][now_user]=int(sorted_data[j,1])
################################################### 根据user2hours记录老用户的时长，
# 因为字典操作需要保证key值存在，所以需要二分搜索key (用户id)是否是在字典的keys中出现过

# id2label=np.zeros([50000])
# id2pro=np.zeros([50000,9])
# id2num=np.zeros([50000,9])

id2label={}
id2pro={}
id2num={}
all_keys=[]  #记录所有训练集用户id，用于后续二分查询
for i in range(9):
    temp=list(user2hours[i].keys())
    temp.sort()
    all_keys.append(temp[:])
files=getfiles("val")
all_cnt=len(files)
print(all_cnt)
cnt=0
acc_cnt=0
last_time=time()
for file in files:
    s=file.split(".txt")[0].split("/")[-1].split("_")
    reg_id=s[0]
    true_label=int(s[1])
    table = pd.read_table(file, header=None)
    table2cnt=table.values[:,:]
    user_num=table2cnt.shape[0]
    for i in range(user_num):
        now_user_hours=table2cnt[i,1].count("&")
        table2cnt[i,1]=now_user_hours
    temp=table2cnt[:,:]
    sorted_data=temp[np.lexsort(temp.T)]
    rate=0.7  #剔除单个测试样本中时长较短的用户，剔除比例为rate
    first_user=int(user_num*rate)
    reg_per_label=np.zeros([9])
    for i in range(first_user,user_num):
        now_user=str(sorted_data[i,0])
        hours_per_label=np.zeros([9])
        for j in range(9):
            if find_key(all_keys[j],now_user):
                hours_per_label[j]=user2hours[j][now_user]
        reg_per_label+=hours_per_label
    id2num[reg_id]=reg_per_label
    pro=cnt2pro(reg_per_label)
    pre_label=np.argmax(pro)+1
    if np.sum(reg_per_label)==0:  #该测试样本内老用户为0，用类别 -1 标记缺失
        pre_label=-1
    id2label[reg_id]=pre_label
    id2pro[reg_id]=pro[:]
    if true_label==pre_label:
        acc_cnt+=1
    cnt+=1
    if cnt%1000==0:
#         print("cnt",cnt)
        now_time=time()
#         print("val acc",acc_cnt/cnt)
        print("%.4f%%"%(cnt/all_cnt*100),"val acc ",acc_cnt/cnt,now_time-last_time)
        last_time=time()
#     cnt+=1

# np.save("id2num_val.npy",id2num)
# np.save("id2label_val.npy",id2label)
# np.save("id2pro_val.npy",id2pro)
files.sort()
f_list=[]
for file in files:
    s=file.split('.txt')[0].split('/')[-1].split('_')[0]
    f_list.append(s)
id2num_val=open('id2num_val.txt','w')
for key in f_list:
    line=key+','+str(id2num[key][0])+','+str(id2num[key][1])+','+str(id2num[key][2])+','+str(id2num[key][3])+','+str(id2num[key][4])+','+str(id2num[key][5])+','+str(id2num[key][6])+','+str(id2num[key][7])+','+str(id2num[key][8])+'\n'
    id2num_val.write(line)
id2num_val.close()
print(1)
id2label_val=open('id2label_val.txt','w')
for key in f_list:
    line=key+','+str(id2label[key])+'\n'
    id2label_val.write(line)
id2label_val.close()
print(2)
id2pro_val=open('id2pro_val.txt','w')
for key in f_list:
    line=key+','+str(id2pro[key][0])+','+str(id2pro[key][1])+','+str(id2pro[key][2])+','+str(id2pro[key][3])+','+str(id2pro[key][4])+','+str(id2pro[key][5])+','+str(id2pro[key][6])+','+str(id2pro[key][7])+','+str(id2pro[key][8])+'\n'
    id2pro_val.write(line)
id2pro_val.close()
print(3)

1 0.0
1 0.005442423890150987
1 0.010884847780301974
1 0.01632727167045296
1 0.021769695560603947
1 0.027212119450754933
1 0.03265454334090592
1 0.03809696723105691
1 0.043539391121207895
1 0.048981815011358884
1 0.05442423890150987
1 0.059866662791660856
1 0.06530908668181185
1 0.07075151057196283
1 0.07619393446211382
1 0.0816363583522648
1 0.08707878224241579
1 0.09252120613256679
1 0.09796363002271777
1 0.10340605391286875
1 0.10884847780301973
1 0.11429090169317073
1 0.11973332558332171
1 0.1251757494734727
1 0.1306181733636237
1 0.13606059725377467
1 0.14150302114392566
1 0.14694544503407664
1 0.15238786892422765
1 0.15783029281437863
1 0.1632727167045296
1 0.1687151405946806
1 0.17415756448483158
1 0.17959998837498256
1 0.18504241226513357
1 0.19048483615528455
1 0.19592726004543554
1 0.20136968393558652
1 0.2068121078257375
1 0.21225453171588848
1 0.21769695560603947
1 0.22313937949619048
1 0.22858180338634146
1 0.23402422727649244
1 0.23946665116664342
1 0.2449090750567944
1 0.

5 0.8124235814068739
6 0.0
6 0.014578045754654406
6 0.029156091509308812
6 0.04373413726396322
6 0.058312183018617625
6 0.07289022877327203
6 0.08746827452792644
6 0.10204632028258084
6 0.11662436603723525
6 0.13120241179188966
6 0.14578045754654406
6 0.16035850330119847
6 0.17493654905585287
6 0.18951459481050728
6 0.2040926405651617
6 0.2186706863198161
6 0.2332487320744705
6 0.2478267778291249
6 0.2624048235837793
6 0.2769828693384337
6 0.2915609150930881
6 0.3061389608477425
6 0.32071700660239694
6 0.3352950523570513
6 0.34987309811170575
6 0.3644511438663601
6 0.37902918962101456
6 0.39360723537566894
6 0.4081852811303234
6 0.42276332688497775
6 0.4373413726396322
6 0.45191941839428657
6 0.466497464148941
6 0.4810755099035954
6 0.4956535556582498
6 0.5102316014129042
6 0.5248096471675586
6 0.539387692922213
6 0.5539657386768674
6 0.5685437844315219
6 0.5831218301861762
6 0.5976998759408306
6 0.612277921695485
6 0.6268559674501394
6 0.6414340132047939
6 0.6560120589594483
6 0.67059

/root/userfolder/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars


2.0000% val acc  0.714 61.50051951408386
4.0000% val acc  0.7185 61.52846074104309
6.0000% val acc  0.712 60.47714376449585
8.0000% val acc  0.70675 63.03462219238281
10.0000% val acc  0.7086 61.20519685745239
12.0000% val acc  0.7073333333333334 61.98867678642273
14.0000% val acc  0.706 58.134509325027466
16.0000% val acc  0.70525 57.26153254508972
18.0000% val acc  0.7075555555555556 58.42552423477173
20.0000% val acc  0.7046 57.56447649002075
22.0000% val acc  0.7057272727272728 59.108797550201416
24.0000% val acc  0.706 57.927088499069214
26.0000% val acc  0.7069230769230769 58.62296795845032
28.0000% val acc  0.7073571428571429 62.80404186248779
30.0000% val acc  0.7066 61.1519079208374
32.0000% val acc  0.7073125 56.327617168426514
34.0000% val acc  0.7058823529411765 61.44174337387085
36.0000% val acc  0.7055555555555556 67.33970642089844
38.0000% val acc  0.7065263157894737 68.50065422058105
40.0000% val acc  0.70725 59.46038293838501


KeyboardInterrupt: 

In [28]:
files=getfiles('val')
files.sort()
f_list=[]
for file in files:
    s=file.split('.txt')[0].split('/')[-1].split('_')[0]
    f_list.append(s)
id2num_val=open('id2num_val.txt','w')
for key in f_list:
    line=key+','+str(id2num[key][0])+','+str(id2num[key][1])+','+str(id2num[key][2])+','+str(id2num[key][3])+','+str(id2num[key][4])+','+str(id2num[key][5])+','+str(id2num[key][6])+','+str(id2num[key][7])+','+str(id2num[key][8])+'\n'
    id2num_val.write(line)
id2num_val.close()
print(1)
id2label_val=open('id2label_val.txt','w')
for key in f_list:
    line=key+','+str(id2label[key])+'\n'
    id2label_val.write(line)
id2label_val.close()
print(2)
id2pro_val=open('id2pro_val.txt','w')
for key in f_list:
    line=key+','+str(id2pro[key][0])+','+str(id2pro[key][1])+','+str(id2pro[key][2])+','+str(id2pro[key][3])+','+str(id2pro[key][4])+','+str(id2pro[key][5])+','+str(id2pro[key][6])+','+str(id2pro[key][7])+','+str(id2pro[key][8])+'\n'
    id2pro_val.write(line)
id2pro_val.close()
print(3)

1
2
3


In [5]:
close_txt(id_cnt_files)